In [1]:
from pymongo import MongoClient
from collections import defaultdict
import datetime
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np

In [2]:
cluster = MongoClient("mongodb://localhost:27017")

db = cluster["smartshark"]

In [3]:
projectCollections = db["project"]

In [4]:
project_kafka = projectCollections.find_one({"name" : "kafka"})
print(project_kafka)

{'_id': ObjectId('5b0fa95e0da3ef431af80833'), 'name': 'kafka'}


In [5]:
pull_request_system_Collection = db['pull_request_system']

In [6]:
pullRequestSystemKafka = pull_request_system_Collection.find_one({"project_id" : project_kafka['_id']})
print(pullRequestSystemKafka)

{'_id': ObjectId('5ff34494cef070edcc19bb6d'), 'project_id': ObjectId('5b0fa95e0da3ef431af80833'), 'url': 'https://api.github.com/repos/apache/kafka/pulls', 'last_updated': datetime.datetime(2021, 1, 6, 19, 51, 16, 785000)}


In [7]:
pull_request_Collection = db['pull_request']

Find all Pull requests belong to kafka project

In [8]:
# developersWithPRcount = pull_request_Collection.aggregate([
#     {"$match" : {"pull_request_system_id" : pullRequestSystemKafka['_id']}},
#     {"$group" : {"_id":"$creator_id", "PRCount":{"$sum":1}}}
# ])

pullRequestsInKafka = pull_request_Collection.find({"pull_request_system_id" : pullRequestSystemKafka['_id']})

# for pullReq in pullRequestsInKafka:
#     print(pullReq)

In [9]:
pullRequestIds_Kafka = set()
for entry in pullRequestsInKafka:
    # print(entry["_id"])
    pullRequestIds_Kafka.add(entry["_id"])

print(pullRequestIds_Kafka)
# print(len(pullRequestIds_Kafka))

{ObjectId('5ff3776b6f08a1699901994f'), ObjectId('5ff40a626f08a16999059452'), ObjectId('5ff6e1dd4c0c286ceffe617b'), ObjectId('5ff7070d4c0c286cefff8396'), ObjectId('5ff70bd44c0c286cefffc6d5'), ObjectId('5ff365536f08a16999010d85'), ObjectId('5ff3c5f46f08a1699903a244'), ObjectId('5ff71adf4c0c286cef005070'), ObjectId('5ff71a9e4c0c286cef004d01'), ObjectId('5ff411566f08a1699905dcd8'), ObjectId('5ff721914c0c286cef006999'), ObjectId('5ff712c84c0c286cefffde32'), ObjectId('5ff37d3a6f08a16999019d91'), ObjectId('5ff6fdd84c0c286cefff5335'), ObjectId('5ff3e4fa6f08a16999048e55'), ObjectId('5ff359d36f08a1699900c986'), ObjectId('5ff34756cef070edcc19d377'), ObjectId('5ff360c96f08a1699900dda1'), ObjectId('5ff3c4796f08a16999038fb1'), ObjectId('5ff3e0d56f08a16999045f31'), ObjectId('5ff718694c0c286cef002aab'), ObjectId('5ff6e0824c0c286ceffe5189'), ObjectId('5ff35a6e6f08a1699900cf47'), ObjectId('5ff3ee4c6f08a1699904b984'), ObjectId('5ff399df6f08a16999024ca9'), ObjectId('5ff381476f08a1699901cab9'), ObjectId('5

In [10]:
pull_request_review_collections = db['pull_request_review']

Find Reviewers and their pull request count

In [11]:
pullRequestReviewerCount = []
for pullReqId in pullRequestIds_Kafka:
    temp = pull_request_review_collections.aggregate([
        {"$match" : {"pull_request_id" : pullReqId}},
        {"$group" : {"_id":"$creator_id", "PRCount":{"$sum":1}}}
    ])

    for entry in temp:
        pullRequestReviewerCount.append([entry["_id"], entry["PRCount"]])

# print(pullRequestReviewerCount)

In [12]:
pullRequestReviewerCount.sort(key=lambda x : x[1], reverse=True)
print(pullRequestReviewerCount)

[[ObjectId('5d92f56b3dfe5ddb856b26ea'), 223], [ObjectId('5d92f56b3dfe5ddb856b26ea'), 167], [ObjectId('5b0fc428065f3902068e0f7c'), 132], [ObjectId('5d92f56b3dfe5ddb856b26ea'), 124], [ObjectId('5ff358f8c26a57681ea701b4'), 121], [ObjectId('5ff375f7c26a57681eafe41a'), 111], [ObjectId('5ff34615c26a57681ea258d1'), 103], [ObjectId('5bc8709857674ee167d80285'), 102], [ObjectId('5c5012d8621a9a77b35fc545'), 98], [ObjectId('5b111d82328aa9d5316b45b4'), 93], [ObjectId('5b0fc420065f3902098e047b'), 92], [ObjectId('5ff375f7c26a57681eafe41a'), 91], [ObjectId('5b0fc400065f39020b8de36a'), 88], [ObjectId('5b2cc398cba9df6755d3056a'), 88], [ObjectId('5d92f56b3dfe5ddb856b26ea'), 84], [ObjectId('5b0fc401065f3902068de3cf'), 83], [ObjectId('5bbf070c57674ee1673024be'), 81], [ObjectId('5ff34615c26a57681ea258d1'), 76], [ObjectId('5ff345ffc26a57681ea252bd'), 76], [ObjectId('5b0fc401065f3902068de3cf'), 75], [ObjectId('5b0fc40c065f39020e8deb99'), 73], [ObjectId('5ff3d4d7c26a57681ec463f5'), 69], [ObjectId('5ff35ae0c26a

In [13]:
developerRankDict = defaultdict(float)

for devId, pr_count in pullRequestReviewerCount:
    developerRankDict[devId] = pr_count / len(pullRequestIds_Kafka)


for entry in developerRankDict.items():
    print(entry)

(ObjectId('5d92f56b3dfe5ddb856b26ea'), 0.00010169836265636123)
(ObjectId('5b0fc428065f3902068e0f7c'), 0.00010169836265636123)
(ObjectId('5ff358f8c26a57681ea701b4'), 0.00010169836265636123)
(ObjectId('5ff375f7c26a57681eafe41a'), 0.00010169836265636123)
(ObjectId('5ff34615c26a57681ea258d1'), 0.00010169836265636123)
(ObjectId('5bc8709857674ee167d80285'), 0.00010169836265636123)
(ObjectId('5c5012d8621a9a77b35fc545'), 0.00010169836265636123)
(ObjectId('5b111d82328aa9d5316b45b4'), 0.00010169836265636123)
(ObjectId('5b0fc420065f3902098e047b'), 0.00010169836265636123)
(ObjectId('5b0fc400065f39020b8de36a'), 0.00010169836265636123)
(ObjectId('5b2cc398cba9df6755d3056a'), 0.00010169836265636123)
(ObjectId('5b0fc401065f3902068de3cf'), 0.00010169836265636123)
(ObjectId('5bbf070c57674ee1673024be'), 0.00010169836265636123)
(ObjectId('5ff345ffc26a57681ea252bd'), 0.00010169836265636123)
(ObjectId('5b0fc40c065f39020e8deb99'), 0.00010169836265636123)
(ObjectId('5ff3d4d7c26a57681ec463f5'), 0.00701718702328

In [14]:
# print(pullRequestIds_Kafka)
pullRequestIds_Kafka_list = list(pullRequestIds_Kafka)
trainingSize = len(pullRequestIds_Kafka)*0.7
testingSize = len(pullRequestIds_Kafka)*0.3

trainingData = []
testingData = []

count = 0
for x in pullRequestIds_Kafka_list : 
    if count < trainingSize:
        trainingData.append(x)
    else:
        testingData.append(x)
    count+=1

print(len(trainingData))
print(len(testingData))


6884
2949


In [34]:
import random 
rankListOrg = []
rankListDummy = []

c = 0
for i in developerRankDict : 
        rankListOrg.append(developerRankDict[i])
        # rankListDummy.append(developerRankDict[i])
rankListOrg.sort(reverse = True)
rankListOrg = rankListOrg[0 : 20]

rankListDummy = rankListOrg.copy()
random.shuffle(rankListDummy)

print(rankListOrg)
print(rankListDummy)

[0.007017187023288925, 0.003864537780941727, 0.003050950879690837, 0.002949252517034476, 0.0024407607037526696, 0.002237363978439947, 0.0020339672531272247, 0.0019322688904708635, 0.0019322688904708635, 0.0017288721651581409, 0.0017288721651581409, 0.0016271738025017796, 0.0015254754398454184, 0.0015254754398454184, 0.001322078714532696, 0.001322078714532696, 0.001322078714532696, 0.001322078714532696, 0.001322078714532696, 0.0012203803518763348]
[0.002949252517034476, 0.001322078714532696, 0.003050950879690837, 0.0016271738025017796, 0.001322078714532696, 0.0020339672531272247, 0.0015254754398454184, 0.0017288721651581409, 0.0012203803518763348, 0.002237363978439947, 0.001322078714532696, 0.0019322688904708635, 0.003864537780941727, 0.007017187023288925, 0.0015254754398454184, 0.001322078714532696, 0.0024407607037526696, 0.0017288721651581409, 0.0019322688904708635, 0.001322078714532696]


In [39]:
print(type(rankListDummy))

<class 'list'>


In [107]:
import rbo
S = [0.007017187023288925, 0.003864537780941727, 0.003050950879690837, 0.002949252517034476,0.0024407607037526696,0.002237363978439947,0.0020339672531272247,0.00193226889047086,0.001932268890470863,0.0017288721651581409,0.00172887216515,0.00162717380250, 0.00152547543,0.001525475, 0.0013220, ]
T = [0.002949252517034476, 0.001322078714532696, 0.003050950879690837, 0.0016271738025017796,0.00132207871453269,0.0020339672531272247,0.0015254754398454184,0.00172887216515814,0.0012203803518763348,0.002237363978439947,0.001322078714532, 0.0019322688904708, 0.003864537780,0.0070171870, 0.00152547543,]

print(type(S))
print(rbo.RankingSimilarity(S, T).rbo())

<class 'list'>
0.31515207015207014


In [41]:
import rbo

arr1 = np.asarray(rankListOrg)
arr2 = np.asarray(rankListDummy)
# score = rb.RankingSimilarity(rankListOrg, rankListDummy).rbo()
# print(score)
print(rbo.RankingSimilarity(arr1,arr2).rbo())

AssertionError: 